# PROPI

In [2]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [3]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [4]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [12]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV4'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 4.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [13]:
# for single frequency XY8-N:
propi = OrderedDict()
propi['mw_amp'] = 0.25
propi['spinlock_amp'] = 0.111
propi['mw_freq'] = 100e6
propi['tau'] = 52.0*1e-6
propi['N'] = 2
propi['M'] = 1000
propi['step_M'] = 1
propi['measurement_time'] = 240*60
propi['refocus_interval'] = 2*60
propi['alternating']=True
propi['rabi_period']=186.5*1e-9

In [9]:
# generate new HH sequence
sequencegeneratorlogic.delete_sequence('propi')    
sequencegeneratorlogic.generate_PROPI('propi', propi['rabi_period'], propi['spinlock_amp'], 100e6, 0.25, 
                                      propi['tau'], propi['N'], propi['M'], propi['step_M'], 'a_ch1', 3.0e-6, 
                                      2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', True)
# sample, upload and load waveform
if 'propi' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of propi in notebook timed out.')
pulsedmasterlogic.sample_sequence('propi', True)

while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [10]:
refocus_poi(setup['poi'])

118340.0 2.278588552806867e-08


True

In [14]:
do_propi_refocus(setup['poi'])

118340.0 9.29861568055266e-09
118340.0 9.22548182994009e-09
118340.0 5.6855908871054775e-09
118340.0 2.2552076503288144e-09
118340.0 2.3466698327135954e-09
2D gaussian fit not successfull
118340.0 0.0
2D gaussian fit not successfull
118340.0 0.0
118340.0 7.660176325751408e-09
118340.0 1.3613718896125456e-08
118340.0 3.698159426666859e-11
118340.0 7.709492707126892e-09
118340.0 6.014635902102229e-09
118340.0 1.7312732931900406e-09
2D gaussian fit not successfull
118340.0 0.0
118340.0 6.498477711794972e-09
118340.0 1.2667594281917844e-09
118340.0 9.246556054096349e-10
118340.0 1.4411064432645507e-09
118340.0 1.2182866808473428e-10
2D gaussian fit not successfull
118340.0 0.0
118340.0 1.5604061999543474e-08
118340.0 4.720873817265611e-09
118340.0 9.409622800108839e-10
118340.0 4.683914070963596e-09
118340.0 2.579798766849813e-10
2D gaussian fit not successfull
118340.0 0.0
118340.0 1.8855002250892826e-08
118340.0 3.1320082781250487e-09
118340.0 4.843315733985432e-09
118340.0 9.57598307965

False

In [7]:
def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.module_state() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_propi(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    m = np.arange(1, propi['M']+1, propi['step_M'])
    # set parameters in analysis tab
    pulsedmasterlogic.measurement_sequence_settings_changed(m, 2*len(m), 100e-6, [], True)
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length'], 2*len(m))
    pulsedmasterlogic.analysis_interval_changed(3)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < propi['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
#     if save_tag=='':
#         pulsedmasterlogic.save_measurement_data('s','SpinLocking-'+str(hh_tau['order'])+'_'+setup['NV_name'],True)
    time.sleep(2)
    return user_terminated

def do_propi_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    propi_total_time = propi['measurement_time']
    propi_runtime = 0.0
    propi['measurement_time'] = propi['refocus_interval']
    end_measure = do_propi('propi', '')
    propi_runtime += propi['refocus_interval']
    while propi_total_time > propi_runtime:
        for i in range(1):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_propi('propi', 'propi')
        propi_runtime += propi['refocus_interval']
        if end_measure:
            break
    propi['measurement_time'] = propi_total_time
    # save measurement
    pulsedmasterlogic.save_measurement_data('s','PROPI'+'_'+'N='+str(propi['N'])+'_' 
                                            +str(propi['spinlock_amp'])+'_'+setup['NV_name'],True, False)
    write_to_logfile('propi_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=propi_runtime)
    time.sleep(2)
    return end_measure